In [ ]:
import h5py
import numpy as np
import swiftsimio
import matplotlib.pyplot as plt

import sys
sys.path.append("./software/HBT-HERONS/toolbox/")
from HBTReader import HBTReader

# Initial conditions

monofonIC saves the power spectrum of fluctuations of your chosen cosmological model. We can load them and plot them to see which scales have the most power. 

In [ ]:
power_spectrum = np.loadtxt("./parameter_files/monofonic/example_input_powerspec.txt")

fig, ax1 = plt.subplots(1)
ax1.plot(power_spectrum[:,0],power_spectrum[:,1])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel('k [h/Mpc]')
ax1.set_ylabel('Ptot')
plt.show()

We can also plot the positions of the particles, which resemble a grid plus some perturbations that reflect the power spectrum.

In [ ]:
with h5py.File("./outputs/monofonic/ics_swift.hdf5") as file:
    coordinates = file["PartType1/Coordinates"][()]
    
fig, ax1 = plt.subplots(1)
ax1.scatter(coordinates[:,0],coordinates[:,1],s=0.1,lw=0,)
ax1.set_aspect('equal')
ax1.set_xlabel('x [Mpc]')
ax1.set_ylabel('y [Mpc]')
plt.show()

# Present-day matter distribution

Integrating the initial conditions with SWIFT allows us to see how matter is distributed at the present-day. We see much larger overdensities compared to the initial conditions, which reflects the effect that gravity had over cosmic time.

In [ ]:
snapshot = swiftsimio.load("./outputs/SWIFT/snap_0049.hdf5")
coordinates = snapshot.dark_matter.coordinates

    
fig, ax1 = plt.subplots(1)
ax1.scatter(coordinates[:,0],coordinates[:,1],s=0.1,lw=0,)
ax1.set_aspect('equal')
ax1.set_xlabel('x [Mpc]')
ax1.set_ylabel('y [Mpc]')
plt.show()

# Bound mass functions

We can use the HBT-HERONS catalogues to see how many subhaloes of a given mass there are in the box.

In [ ]:
# Access all the HBT-HERONS catalogues
subhalo_catalogues = HBTReader("./outputs/HBT-HERONS/")

# Load the last snapshot of the simulation
subhaloes = subhalo_catalogues.LoadSubhalos()
subhalo_mbound = subhaloes["Mbound"] * 1e10 # Msun
subhalo_trackid = subhaloes["TrackId"]

fig, ax1 = plt.subplots(1)
ax1.plot(np.sort(mbound)[::-1], np.arange(len(mbound))+1)
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.set_xlabel('Bound mass [Msun]')
ax1.set_ylabel('N(>Bound Mass)')
plt.show()

We can also see the mass evolution of the most massive subhalo in the box, which has a unique identifier (`TrackId`)

In [ ]:
TrackId_to_follow = subhalo_trackid[subhalo_mbound.argmax()]
subhalo_evolution = subhalo_catalogues.GetTrackEvolution(TrackId_to_follow)

fig, ax1 = plt.subplots(1)
ax1.plot(subhalo_evolution["Snapshot"],subhalo_evolution["Mbound"] * 1e10)
ax1.set_yscale('log')
ax1.set_ylabel('Bound mass [Msun]')
ax1.set_xlabel('Snapshot')
plt.show()

# SOAP catalogues - Vmax functions

SOAP takes the HBT-HERONS subhalo centres and particles to measure many properties of the subhalo population. It is unit-aware and fully compatible with swiftsimio, making it easy to circumvent unit-related-migraines.

In [ ]:
SOAP_catalogue = swiftsimio.load("./outputs/SOAP/SOAP_uncompressed/HBTplus/halo_properties_0049.hdf5")

fig, ax1 = plt.subplots(1)
ax1.plot(np.sort(SOAP_catalogue.bound_subhalo.maximum_circular_velocity.to("km/s"))[::-1],np.arange(len(SOAP_catalogue.bound_subhalo.maximum_circular_velocity))+1)
ax1.set_xscale("log")
ax1.set_yscale("log")
ax1.set_ylabel("N>(Vmax)")
ax1.set_xlabel("Vmax [km/s]")
plt.show()